# BAGGING: _**<u>Bootstrap Aggregation</u>**_
**Librerias a utilizar:**

In [1]:
import pandas as pd
from sklearn import tree

In [2]:
df = pd.read_csv('churn_cleaned.csv')
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,0,0,42,2,0.00,1,1,1,101348.88,1
1,608,1,0,41,1,83807.86,1,0,1,112542.58,0
2,502,0,0,42,8,159660.80,3,1,0,113931.57,1
3,699,0,0,39,1,0.00,2,0,0,93826.63,0
4,850,1,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,0,1,39,5,0.00,2,1,0,96270.64,0
9996,516,0,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,0,0,36,7,0.00,1,0,1,42085.58,1
9998,772,2,1,42,3,75075.31,2,1,0,92888.52,1


In [3]:
def boostrap_sample(df, n, p):
    boostrap = []
    for b in range(0, n):
        # replace True permite que haya items que se repitan en el df
        boostrap.append(df.sample(frac=p, replace=True))
    return boostrap

In [4]:
b_sample = boostrap_sample(df, 3, 0.5)
b_sample

[      CreditScore  Geography  Gender  Age  Tenure    Balance  NumOfProducts  \
 1549          652          0       1   48       4   59486.31              1   
 829           577          0       0   39      10       0.00              2   
 6642          589          0       0   37       6  138497.84              1   
 2131          614          0       1   38       2  116248.88              1   
 5528          645          0       1   32       4       0.00              2   
 ...           ...        ...     ...  ...     ...        ...            ...   
 1011          719          1       1   35       3  122964.88              1   
 4585          603          0       1   30       8       0.00              2   
 5962          579          2       1   42       0  144386.32              1   
 8641          604          2       0   47       4  118907.60              1   
 7269          655          2       1   34       7  118028.35              1   
 
       HasCrCard  IsActiveMember  Esti

In [5]:
def voting(lst):
    one = lst.count(1)
    zero = lst.count(0)
    if one > zero:
        return 1
    return 0

In [6]:
def bagging_predict(b_sample, x, target):
    # almacenar predicciones
    pred = []
    for b in range(0, len(b_sample)):
        # create X and y value
        X = b_sample[b].drop([target], axis=1)
        y = b_sample[b][target]
        # train decision tree
        clf = tree.DecisionTreeClassifier().fit(X,y)
        # append the prediction in the array
        pred.append(clf.predict(x)[0])
    return voting(pred)

In [7]:
def count_hits(values, df, target):
    count = 0
    for key in values:
        values[key] = voting(values[key])
        if key in df[target].index.values and values[key] == df[target][key]:
            count += 1
    return count

In [8]:
def bagging_score(b_sample, df, target):
    # almacenar predicciones
    pred = []
    dfs = {}
    for b in range(0, len(b_sample)):
        df_test = df.drop(b_sample[b].index.values)
        # create X and y value
        X = b_sample[b].drop([target], axis=1)
        y = b_sample[b][target]
        # create X test
        X_test = df_test.drop([target], axis=1)
        # train decision tree
        clf = tree.DecisionTreeClassifier().fit(X,y)
        pred = clf.predict(X_test)
        # iterar sobre las predicciones y agregarlas al diccionario
        # si ya esta el indice, concatenar
        # si no, agregar
        for i in range(0, len(pred)):
            if X_test.index.values[i] in dfs:
                dfs[X_test.index.values[i]].append(pred[i])
            else:
                dfs[X_test.index.values[i]] = [pred[i]]
    #debemos aplicar voting sobre cada key del arreglo
    score = count_hits(dfs, df, target)
    return score / len(dfs)

In [9]:
# create 100 boostrap samples with 50% original size each one
b_sample = boostrap_sample(df, 3, 0.5)

In [10]:
x = [[500, 0, 1, 40, 1, 5000, 1, 1, 0, 100000]]
# realizaremos 100 muestras boostrap y cada muestra sera del 50% del tamaño del df original
pred = bagging_predict(b_sample, x, 'Exited')
print("CreditScore, Geography, Gender, Age, Tenure, Balance, NumOfProducts, HasCrCard, isActiveMember, EstimatedSalary")
print(x[0])
if pred == 1:
    print("We predict an EXIT of the client")
else:
    print("We predict client will REMAIN in the bank")

CreditScore, Geography, Gender, Age, Tenure, Balance, NumOfProducts, HasCrCard, isActiveMember, EstimatedSalary
[500, 0, 1, 40, 1, 5000, 1, 1, 0, 100000]
We predict client will REMAIN in the bank


/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


In [11]:
bagging_score(b_sample, df, 'Exited')

0.8188097768331563

In [12]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,0,0,42,2,0.00,1,1,1,101348.88,1
1,608,1,0,41,1,83807.86,1,0,1,112542.58,0
2,502,0,0,42,8,159660.80,3,1,0,113931.57,1
3,699,0,0,39,1,0.00,2,0,0,93826.63,0
4,850,1,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,0,1,39,5,0.00,2,1,0,96270.64,0
9996,516,0,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,0,0,36,7,0.00,1,0,1,42085.58,1
9998,772,2,1,42,3,75075.31,2,1,0,92888.52,1


In [13]:
from sklearn.ensemble import RandomForestClassifier
X = df.drop(['Exited'], axis=1)
y = df['Exited']
random_forest = RandomForestClassifier(n_estimators = 3, bootstrap= True).fit(X, y)
y = random_forest.predict_proba([[500, 0, 1, 40, 1, 5000, 1, 1, 0, 100000]])
y

/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


array([[0.66666667, 0.33333333]])